In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
import re
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt




In [ ]:
def train_model(df):

    def create_sequences(data, n_steps):
        X = []
        y = []
        for i in range(len(data)-n_steps):
            X.append(data.iloc[i:i+n_steps, :].values)
            y.append(data.iloc[i+n_steps, 0])
        return np.array(X), np.array(y), np.array

    # create sequences
    n_steps = 7
    X, y, df = create_sequences(df, n_steps)
    X.shape, y.shape

    # split into train and test sets
    n_train = 365*5
    X_train, X_test, y_train, y_test = X[:n_train], X[n_train:], y[:n_train], y[n_train:]
    #create validation set
    X_train, X_val, y_train, y_val = X_train[:-7], X_train[-7:], y_train[:-7], y_train[-7:]

    model_3 = Sequential()
    model_3.add(LSTM(32, activation='relu', input_shape=(n_steps, n_features)))
    model_3.add(Dropout(0.2))
    model_3.add(Dense(8, activation='relu'))
    model_3.add(Dropout(0.2))
    model_3.add(Dense(1))

    model_3.compile(optimizer=tf.keras.optimizers.Adam(beta_1 = 0.999), loss='mse')

    history3 = model_3.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), verbose=1)
    plt.plot(history3.history['loss'], label='train')
    plt.plot(history3.history['val_loss'], label='test')
    plt.legend()
    plt.show()

    yhat3 = model_3.predict(X_test, verbose=0)
    # denormalize yhat3
    yhat3 = scaler.inverse_transform(yhat3)

    plt.plot(y_test_denorm, label='Test')
    plt.plot(yhat3, label='Predicted')
    plt.legend()

    yhat_train3 = model_3.predict(X_train, verbose=0)
    yhat_train3 = scaler.inverse_transform(yhat_train3)
    rmse3 = sqrt(mean_squared_error(y_train_denorm, yhat_train3, squared=False))
    print('Train RMSE: %.3f' % rmse3)

    rmse3_test = sqrt(mean_squared_error(y_test_denorm, yhat3, squared=False))
    print('Test RMSE: %.3f' % rmse3_test)

    #save model_3 in models folder with keras
    filename = '../src/models/modelo_transito.h5'
    model_3.save(filename)